In [1]:
import os
import logging
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# class ForecasterDev():
    
#     def __init__(self, params={}, data=None, fcList=[]):
#         '''
#         Initializes the framework with a list of models to 
#         evaulate, and a training dataset to use for model
#         training and testing.
        
#         inputs:
#         fcList: list of dicts defining forecaster models
#             to evaluation. Dicts contain keys:
            
#             fun: name of forecast model to evaluate, 
#             corresponding to model types defined in fcLib
            
#             name: (optional) string with name of forecast model
#             being used. Necessary if passing multiple forecasters
#             of same type with different parameters. Default name
#             set to 'fun' name. If multple forecasters of same type
#             passed, the 'name' will be auto incrememted
            
#             parameters: parameters to tune each model.
#         data: dict with X and y items, corresponding to features
#             and target values. X should be pandas df, y numpy array

#         '''
        
#         self.params = params
#         self.fcList = fcList
        
#         self.dataDf = None
#         self.backupDate = datetime.now()
        
#         self.fcData = None
#         self.bestModel = None
#         self.validData = False
        
        
#         '''
#         self.dtFormat = None
#         self.addNewCols = True
#         self.showWarnings = False
        
#         '''
        
#         # parse parameter
#         k = self.params.keys()
#         self.train_size = self.params['train_size'] if 'train_size' in k else 0.75
#         self.train_method = self.params['train_method'] if 'train_method' in k else 'train_test_split'
#         self.min_days = self.params['min_days'] if 'min_days' in k else 14
#         self.colList = self.params['colList'] if 'colList' in k else []
#         self.targetColName = self.params['targetColName'] if 'targetColName' in k else 'target'
#         self.recordPath = self.params['recordPath'] if 'recordPath' in k else './data.csv'
#         self.backupInterval = self.params['backupInterval'] if 'backupInterval' in k else timedelta(days=7)
        
#         # check that backupInterval type is timedelta
#         if not type(self.backupInterval) == type(timedelta(days=1)):
#             self.backupInterval = timedelta(days=7)
        
#         if data:
#             self.updateData(data)
#         else:
#             self.initRecord()
            

#     def unpackRecord(self):
#         '''
#         method unpacks record df into X and y using self.targetColName
#         '''
        
#         # init data attr to empty
#         self.data = {}
#         self.X = None
#         self.y = None
        
#         # if no data stored, do nothing
#         if self.dataDf is None:
#             return
        
#         if self.dataDf.shape[0] == 0:
#             return

#         # get list of all columns
#         dfColsList = list(self.dataDf.columns)

#         # check if targetCol is currently in df
#         if self.targetColName not in dfColsList:
#             # if target is not in list of column, use last column from df
#             logging.warning(f"Target column {self.targetColName} not found in data, using {dfColsList[-1]}")
#             targetColName = dfColsList[-1]
#         else:
#             targetColName = self.targetColName

#         # divide data based on current targetColName
#         dfColsList.remove(targetColName)
#         columnsX = dfColsList
#         columnsY = targetColName

#         self.data = {
#             'X': self.dataDf[columnsX],
#             'y': self.dataDf[columnsY]
#         }

#         self.X = self.dataDf[columnsX],
#         self.y = self.dataDf[columnsY]


#     def initRecord(self):
#         '''
#         method initializes record dataframe. Creates empty datafram
#         '''
        
#         colList = self.colList

#         # remove timestamp from list, which is used as index
#         if 'timestamp' in colList:
#             colList.remove('timestamp')

#         # initialize df
#         df = pd.DataFrame(columns = colList)

#         self.dataDf = df

#         # unpack current dataDf to X and y attributes
#         self.unpackRecord()
    
    
#     def addRecord(self, newRecord):
#         '''
#         method adds new record (row) to dataDf
        
#         newRecord - dict where keys should match dataDf col names
#         must include field 'timestamp' for df index
#         '''
        
#         # stop if required field 'timestamp' is missing
#         if 'timestamp' not in newRecord.keys():
#             logging.error('could not add record. Timestamp field missing')
#             return
        
            
#         # separate timestamp from other fields    
#         timestamp = newRecord.pop('timestamp') 
        
#         # check if there are new fields in record not found in self.dataDf
#         dfColumns = self.dataDf.columns
        
#         for field in newRecord.keys():
#             if field not in dfColumns:
#                 # if field missing, add to df with None type as default value
#                 self.dataDf[field] = None
        
#         self.dataDf.loc[timestamp] = newRecord
        
#         # unpack current dataDf to X and y attributes
#         self.unpackRecord()
    
    
#     def saveRecord(self):
#         '''
#         method saves current self.dataDf to self.recordPath file as csv
#         '''
        
#         try:
#             self.dataDf.to_csv(self.recordPath)
            
#             self.backupDate = datetime.now()
            
#         except Exception as e:
#             traceback.print_exc()
#             logging.error(f'Could not write training data to file {self.recordPath}')
        
    
#     def readRecord(self):
#         '''
#         method reads data from self.recordPath and unpacks to X and y attributes
#         '''
#         try:
#             dataDf = pd.read_csv(self.recordPath, index_col = 0)
#             dataDf.index = pd.to_datetime(dataDf.index) 

#             self.dataDf = dataDf
            
#             # unpack current dataDf to X and y attributes
#             self.unpackRecord()
        
#         except Exception as e:
#             traceback.print_exc()
#             logging.error(f'Could not read training data from file {self.recordPath}')
         
        
#     def checkRecordBackup(self):
#         '''
#         method checks if data backup should be perfomed based on self.backupInterval and last backupDate
#         '''
        
#         curTime = datetime.now()
        
#         if (curTime - self.backupDate) > self.backupInterval:
#             logging.info('Saving training data to file')
#             self.saveRecord()
        
        

In [3]:
import sys

cwd = os.getcwd()
folder = os.path.join(cwd, '..', 'src')
 
# adding Folder_2 to the system path
sys.path.insert(0, folder)

from fcSelector import ForecasterFramework

### Create new selector with data back-up parameters

In [4]:
parameter = {
    'targetColName': 't',
#     'colList': ['c1', 'c2', 'c3', 't1', ],
    'recordPath': './training_data.csv',
    'backupInterval': 5
}

x = ForecasterFramework(params = parameter)

### Add individual data records as dicts

In [5]:
# newRecords = [



newRecords = [
    {
        '2018-07-21 00:00:00': {
            'c1': 11,
            'c2': 22,
            'c3': 1,
            't': 101,
            'ts1': [1,2,3,4,5]
        }
    },
    {
        '2018-07-21 01:00:00': {
            'c1': 13,
            'c2': 25,
            'c3': 2,
            't': 103,
            'ts1': [11,2,3,4,5]
        }
    },
    {
        '2018-07-21 02:00:00': {
            'c1': 15,
            'c2': 35,
            'c3': 4,
            't': 112,
            'ts1': [12,2,3,4,5]
        }
    },
    {
        '2018-07-21 03:00:00': {
            'c1': 16,
            'c2': 44,
            'c3': 5,
            't': 124,
            'ts1': [13,2,3,4,5]
        }
    },
    {
        '2018-07-21 04:00:00': {
            'c1': 17,
            'c2': 49,
            'c3': 6,
            't': 144,
            'ts1': [14,2,3,4,5]
        }
    }
]

for newRecord in newRecords:

    x.addRecord(newRecord)


2022-10-25 11:15:15 WARNING  Indexed-field ts1 length is 5. expected 24


### Run backup method to save data if interval has passed

In [6]:
type(x.y)

pandas.core.series.Series

In [7]:
cList = list(x.dataDf.columns)[:-1]

x.dataDf[cList]

,c1,c2,c3,t
2018-07-21 00:00:00,11,22,1,101
2018-07-21 01:00:00,13,25,2,103
2018-07-21 02:00:00,15,35,4,112
2018-07-21 03:00:00,16,44,5,124
2018-07-21 04:00:00,17,49,6,144


In [8]:
x.X.shape

(5, 4)

In [9]:
list(newRecords[0].keys())

['2018-07-21 00:00:00']

In [10]:
recordPath = './training_data.csv'

# set params for data record
parameter = {
    'targetColName': 't',
#     'colList': ['c1', 'c2', 'c3'],
    'recordPath': recordPath,
    'backupInterval': '5'
}

# create 2 empty forecaster frameworks
a = ForecasterFramework(params = parameter)
b = ForecasterFramework(params = parameter)

# define example training data to add
newRecords = [
    {
        '2018-07-21 00:00:00': {
            'c1': 11,
            'c2': 22,
            'c3': 1,
            't': 101,
            'ts1': [1,2,3,4,5]
        }
    },
    {
        '2018-07-21 01:00:00': {
            'c1': 13,
            'c2': 25,
            'c3': 2,
            't': 103,
            'ts1': [11,2,3,4,5]
        }
    },
    {
        '2018-07-21 02:00:00': {
            'c1': 15,
            'c2': 35,
            'c3': 4,
            't': 112,
            'ts1': [12,2,3,4,5]
        }
    },
    {
        '2018-07-21 03:00:00': {
            'c1': 16,
            'c2': 44,
            'c3': 5,
            't': 124,
            'ts1': [13,2,3,4,5]
        }
    },
    {
        '2018-07-21 04:00:00': {
            'c1': 17,
            'c2': 49,
            'c3': 6,
            't': 144,
            'ts1': [14,2,3,4,5]
        }
    }
]


# add records to framework a
for newRecord in newRecords:

    a.addRecord(newRecord)

# save record to file
a.saveRecord()

# read stored data into framework b
b.readRecord()

# delete external data record file 
if os.path.exists(recordPath):
    os.remove(recordPath)

2022-10-25 11:15:15 WARNING  Indexed-field ts1 length is 5. expected 24


In [11]:
print(a.X.shape)
print(a.y.size)
print(b.X.shape)
print(b.y.size)

(5, 4)
5
(5, 4)
5


In [12]:
x.backupInterval

5

In [13]:
x.checkRecordBackup()

In [14]:
x.dataDf.head()

,c1,c2,c3,t,ts1
2018-07-21 00:00:00,11,22,1,101,NaN
2018-07-21 01:00:00,13,25,2,103,NaN
2018-07-21 02:00:00,15,35,4,112,NaN
2018-07-21 03:00:00,16,44,5,124,NaN
2018-07-21 04:00:00,17,49,6,144,NaN


In [15]:
x.dataMeta

{'c1': {'tsIndexed': False},
 'c2': {'tsIndexed': False},
 'c3': {'tsIndexed': False},
 't': {'tsIndexed': False},
 'ts1': {'tsIndexed': True, 'tsCount': 5}}

In [16]:
type([1,2,3]) is list

True